# Create tables with p-value and change in medians for multiple cancers 

Create a dataframe with p-value results from t-tests for all proteins (trans gene proteomics when PTEN has cnv deletions compared to PTEN wildtype). The dataframe also includes the change in medians between deletions and wildtype. Prepare these tables for further analysis by creating csv files.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

In [2]:
# Returns a dataframe with proteomics and mutation type

# all_prot: list of trans genes

def all_prot_format_df(cancer_object, all_prot, gene_in = 'PTEN'):
    mut_type = cancer_object.get_genotype_all_vars(gene_in)
    
    if cancer_object is not l:
        # Keep only tumor samples from proteomics
        prot_and_mutations = cancer_object.join_omics_to_mutations(
            mutations_genes = [gene_in], omics_df_name = 'proteomics', omics_genes = all_prot)
        # Reduce a multiindex 
        if isinstance(prot_and_mutations.keys(), pd.core.indexes.multi.MultiIndex):
            prot_and_mutations = cancer_object.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
        prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples

        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        mut_type = mut_type[['Mutation']] 
        prot_df = prot_and_mutations.iloc[:,:-4] # Keep only proteomics
        merged = prot_df.join(mut_type)
        
        # Keep only Wildtype and deletion
        compare = ['Wildtype_Tumor','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
    
    
    # Luad has no somatic mutations for PTEN which changes some things
    else: 
        # get_genotype_all_vars add cnv data under the column PTEN
        mut_type = mut_type.drop(columns= gene_in)
        # different code because no somatic mutation data for pten (can't join to somatic mutations)
        omics = l.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1= gene_in, 
            genes2= all_prot)
        omics = l.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
        omics = omics.drop(columns='PTEN_CNV')
        # Get only tumor samples
        p = l.get_proteomics(tissue_type='tumor')
        tumor_ids = list(p.index)
        get = omics.index.isin(tumor_ids)
        omics = omics[get]
        # Merge Mutation column from get_genotype_all_vars (includes cnv) with proteomics
        merged = omics.join(mut_type) 
        # Keep only No_Mutation (wildtype) and deletion
        compare = ['No_Mutation','Deletion']
        get = merged['Mutation'].isin(compare)
        del_wt = merged[get]
        del_wt['Mutation'] = np.where(
            del_wt['Mutation'] == 'No_Mutation', 'Wildtype_Tumor', 'Deletion')

    return del_wt


In [3]:
'''
Params
cancer_object: Object. Variable for the loaded cancer data set.
cancer_name: Str. name to add to the created dataframe.
del_wt_df: DataFrame. Only samples with deletions and wildtype for PTEN. 
all_prot_list: List. All proteins in proteomics data frame. 

Returns a dataframe with the difference in medians between proteomics with PTEN wt and del (wt - del). 
'''

def get_change_in_medians_df(cancer_object, cancer_name, del_wt_df, all_prot_list):
    d = del_wt_df[del_wt_df.Mutation == "Deletion"]
    wt = del_wt_df[del_wt_df.Mutation == "Wildtype_Tumor"]
    del_med = d.median()
    wt_med = wt.median()

    med_dict = {}
    

    for prot in all_prot_list:
        dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
        med_dict[prot+'_proteomics'] = dif
        
    df = pd.DataFrame.from_dict(med_dict, orient='index', columns=[cancer_name+'_Median'])
    df = df.reset_index().rename(columns={'index':'Proteomics'})
    
    return df
        

#  Step 1: Create Data frames with p-values and differences in median

Each cancer needs a data frame containing only samples that have PTEN cnv deletions and PTEN wildtype with trans proteomics. Use wrap_ttest to run many T-tests for all genes in the proteomics data frame. Use get_change_in_medians_df to create the data frame with change in median values. Merge both data frames. 

Load in cancer data sets from cptac. 

In [4]:
g = cptac.Gbm()
en = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
ls = cptac.Lscc()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

KeyboardInterrupt: 

# Gbm

Part 1: Format data frame using all_prot_format_df. This returns a dataframe with proteomics and a Mutation column containing only PTEN cnv deletions and wildtype tumors. 

In [ ]:
gene = 'PTEN'
g_prot = g.get_proteomics()
g_prot_list = list(g_prot.columns)

g_del_wt = all_prot_format_df(g, g_prot_list)
g_del_wt.head()

Part 2: Run T-tests. Create a data frame of just significant comparisons and another data frame for all comparisons. A moderately stringent correction is used (FDR_BH).

In [ ]:
cols = list(g_del_wt.columns[:-1])

# Get only sig genes
g_sig = u.wrap_ttest(g_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
# Create list of sig genes
if g_sig is not None:
    g_sig_list = list(g_sig.Comparison)
else: 
    g_sig_list = None
print('significant pvals: \n',g_sig)

# Get all pvals
g = u.wrap_ttest(g_del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
g_pval = g.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})

Part 3: Create the differences in median df. (median of wildtype tumors - median of cnv deletions)

In [ ]:
g_med = get_change_in_medians_df(g, "Gbm", g_del_wt, g_prot_list)

Part 4: Merge the p-values and the differences in median dfs.

In [ ]:
g_merged = g_pval.merge(g_med, on='Proteomics',how='outer')
g_merged

# Repeat for other cancers.

# Ovarian

In [ ]:
gene = 'PTEN'
o_prot = o.get_proteomics()
o_prot = o.reduce_multiindex(o_prot, levels_to_drop = 1)
o_prot_list = list(o_prot.columns)

o_del_wt = all_prot_format_df(o, o_prot_list)
# Drop duplicate columns - FIX
o_del_wt = o_del_wt.loc[:,~o_del_wt.columns.duplicated()]

In [ ]:
cols = list(o_del_wt.columns[:-1])

# Get only sig sites
o_sig = u.wrap_ttest(o_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if o_sig is not None:
    o_sig_list = list(o_sig.Comparison)
else: 
    o_sig_list = None
print('significant pvals: \n',o_sig)

# Get all pvals
o = u.wrap_ttest(o_del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
o_pval = o.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Ov_P_Value'})

In [ ]:
o_med = get_change_in_medians_df(o, "Ov", o_del_wt, o_prot_list)

In [ ]:
o_merged = o_pval.merge(o_med, on='Proteomics',how='outer')
o_merged

# Breast

In [ ]:
gene = 'PTEN'
b_prot = b.get_proteomics()
b_prot = b.reduce_multiindex(b_prot, levels_to_drop = 1)
b_prot_list = list(b_prot.columns)

b_del_wt = all_prot_format_df(b, b_prot_list)
b_del_wt = b_del_wt.loc[:,~b_del_wt.columns.duplicated()]
b_del_wt.head()

In [ ]:
cols = list(b_del_wt.columns[:-1])

# Get only sig sites
b_sig = u.wrap_ttest(b_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if b_sig is not None:
    b_sig_list = list(b_sig.Comparison)
else: 
    b_sig_list = None
print('significant pvals: \n',b_sig)

# Get all pvals
b_pval = u.wrap_ttest(b_del_wt, 'Mutation', cols, return_all = True)
b_pval = b_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Brca_P_Value'})

In [ ]:
b_med = get_change_in_medians_df(b, "Brca", b_del_wt, b_prot_list)

In [ ]:
b_merged = b_pval.merge(b_med, on='Proteomics',how='outer')
b_merged

# Colon

In [ ]:
gene = 'PTEN'
c_prot = col.get_proteomics()
c_prot_list = list(c_prot.columns)

c_del_wt = all_prot_format_df(col, c_prot_list)
c_del_wt = c_del_wt.loc[:,~c_del_wt.columns.duplicated()]

In [ ]:
cols = list(c_del_wt.columns[:-1])

# Get only sig sites
c_sig = u.wrap_ttest(c_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if c_sig is not None:
    c_sig_list = list(c_sig.Comparison)
else: 
    c_sig_list = None
print('significant pvals: \n',c_sig)

# Get all pvals
c_pval = u.wrap_ttest(c_del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
c_pval = c_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Colon_P_Value'})

In [ ]:
c_med = get_change_in_medians_df(col, "Colon", c_del_wt, c_prot_list)

In [ ]:
c_merged = c_pval.merge(c_med, on='Proteomics',how='outer')
c_merged

# Hnscc

In [ ]:
gene = 'PTEN'
h_prot = h.get_proteomics()
h_prot_list = list(h_prot.columns)

h_del_wt = all_prot_format_df(h, h_prot_list)
h_del_wt.head()

In [ ]:
cols = list(h_del_wt.columns[:-1])

# Get only sig sites
h_sig = u.wrap_ttest(h_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if h_sig is not None:
    h_sig_list = list(h_sig.Comparison)
else: 
    h_sig_list = None
print('significant pvals: \n',h_sig)

# Get all pvals
h_pval = u.wrap_ttest(h_del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
h_pval = h_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Hnscc_P_Value'})

In [ ]:
h_med = get_change_in_medians_df(h, "Hnscc", h_del_wt, h_prot_list)

In [ ]:
h_merged = h_pval.merge(h_med, on='Proteomics',how='outer')
#del_wt[['PREX2_proteomics','Mutation']].dropna() # in median, not in pval (only 1 value for deletion)
h_merged

# Luad

In [ ]:
gene = 'PTEN'
l_prot = l.get_proteomics()
l_prot = l.reduce_multiindex(l_prot, levels_to_drop = 1)
l_prot_list = list(l_prot.columns)

l_del_wt = all_prot_format_df(l, l_prot_list)
l_del_wt = l_del_wt.loc[:,~l_del_wt.columns.duplicated()]
l_del_wt.head()

In [ ]:
cols = list(l_del_wt.columns[:-2])

# Get only sig sites
l_sig = u.wrap_ttest(l_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if l_sig is not None:
    l_sig_list = list(l_sig.Comparison)
else: 
    l_sig_list = None
print('significant pvals: \n',l_sig)

# Get all pvals
l_pval = u.wrap_ttest(l_del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
l_pval = l_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Luad_P_Value'})
# isoforms for some proteins

In [ ]:
#Isoform code - fix later
#del_wt.replace(to_replace = r'_NP_.*$', value = '', regex = True)

In [ ]:
l_med = get_change_in_medians_df(l, "Luad", l_del_wt, l_prot_list)

In [ ]:
l_merged = l_pval.merge(l_med, on='Proteomics',how='outer')
l_merged

# Lscc

In [ ]:
gene = 'PTEN'
ls_prot = ls.get_proteomics()
ls_prot = ls.reduce_multiindex(ls_prot, levels_to_drop = 1)
ls_prot_list = list(ls_prot.columns)

ls_del_wt = all_prot_format_df(ls, ls_prot_list)
ls_del_wt = ls_del_wt.loc[:,~ls_del_wt.columns.duplicated()]

In [ ]:
cols = list(ls_del_wt.columns[:-2])

# Get only sig sites
ls_sig = u.wrap_ttest(ls_del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if ls_sig is not None:
    ls_sig_list = list(ls_sig.Comparison)
else: 
    ls_sig_list = None
print('significant pvals: \n',ls_sig)

# Get all pvals
ls_pval = u.wrap_ttest(ls_del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
ls_pval = ls_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Lscc_P_Value'})
#ls_pval # isoforms for some proteins

In [ ]:
ls_med = get_change_in_medians_df(ls, "Lscc", ls_del_wt, ls_prot_list)

In [ ]:
ls_merged = ls_pval.merge(ls_med, on='Proteomics',how='outer')
ls_merged

# Endometrial

The Endometrial data set does not have enough cnv deletions to perform a t-test, however the data set does have enough truncation type mutations (nonsense and frame shifts). Different code is needed to create the data frame for Endometrial.

In [ ]:
prot = en.get_proteomics()
p = list(prot.columns)

mut_type = en.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = en.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = p)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')

In [ ]:
cols = list(trunc_wt.columns[:-1])

# Get only sig sites
e_sig = u.wrap_ttest(trunc_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if e_sig is not None:
    e_sig_list = list(e_sig.Comparison)
else: 
    e_sig_list = None
print('significant pvals: \n',e_sig)

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'En_P_Value'})

Differences in median with adaption to trunctation mutations.

In [ ]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in p:
    dif = wt_med[prot+'_proteomics'] - trunc_med[prot+'_proteomics']
    en_d[prot+'_proteomics'] = dif

en_med = pd.DataFrame.from_dict(en_d, orient='index', columns=['En_Median'])
en_med = en_med.reset_index().rename(columns={'index':'Proteomics'})

In [ ]:
e_merged = e_pval.merge(en_med, on='Proteomics',how='outer')
e_merged

# Step 2: Merge all cancer data frames and create csv files

Merge the data frames with p-values and changes in median into one large data frame.

In [ ]:
df1 = g_merged.merge(h_merged, on='Proteomics',how='outer')
df2 = df1.merge(l_merged, on='Proteomics',how='outer')
df3 = df2.merge(ls_merged, on='Proteomics',how='outer')
df4 = df3.merge(b_merged, on='Proteomics',how='outer')
df5 = df4.merge(o_merged, on='Proteomics',how='outer')
df6 = df5.merge(e_merged, on='Proteomics',how='outer')
all_df = df6.merge(c_merged, on='Proteomics',how='outer')
all_df


Create a csv file with all the data for all the proteins.

In [ ]:
all_df.to_csv('all_pval_all_proteins.csv')

# create csv with significant proteins in at least 1 cancer 

In [ ]:
# Keep only genes significant in > 1 cancer
all_sig = []

sig_lists = [g_sig_list, e_sig_list, b_sig_list, o_sig_list, c_sig_list, h_sig_list, l_sig_list, ls_sig_list]

for next_list in sig_lists:
    if next_list is not None:
        all_sig.append(next_list)
        

flat_list = [item for sublist in all_sig for item in sublist] #change list of lists, to just one list
sig = set(flat_list) # remove duplicates

bool_df = all_df['Proteomics'].isin(sig)
sig_df = all_df[bool_df]

sig_df = sig_df.replace(to_replace = r'_proteomics$', value = '', regex = True)
sig_df = sig_df.set_index('Proteomics')
sig_df.head()

In [ ]:
sig_df.to_csv('sig_pval_all_proteins.csv')

Breakdown of significant genes in each cancer

In [ ]:
# see sig in cancer
cancer = ['Gbm','En','Brca','Ov','Colon','Hnscc','Luad']
i = 0
for next_list in sig_lists:

    print(cancer[i], ':')
    if next_list is not None: 
        print(len(next_list),'\n')
    #print(next_list, '\n')
    if (i < 6):
        i += 1
    

# create csv with long data frame to use with HeatMap function

In [ ]:
# Create long df for heat map

cancer = ['Gbm','Hnscc','Luad','Lscc','Brca','Ov','En','Colon']
merged_dfs = [g_merged,h_merged,l_merged,ls_merged,b_merged,o_merged,e_merged,c_merged]

merged = pd.DataFrame()
i = 0
for c in cancer:
    m = merged_dfs[i]
    m2 = m.assign(Cancer = c)
    m2 = m2.rename(columns={c+'_P_Value': 'P_Value'})
    m2 = m2.rename(columns={c+'_Median': 'Medians'})
    merged = merged.append(m2) 
    if i < 7:
        i += 1

# Keep genes with at least one sig ttest
bool_df2 = merged['Proteomics'].isin(sig)
plot_df = merged[bool_df2]
plot_df = plot_df.replace(to_replace = r'_proteomics$', value = '', regex = True)

Create a the size column to use with the HeatMap function for the circle size variable. Because more significant p-values are smaller, an inverse log is used to create a bigger circle size for the more significant p-values. Next the logged p-value is divided by ten to create reasonably sized circles. 

In [ ]:
# log p-vals for right scale in plot (bigger circle, smaller pval)
plot_df['size'] = plot_df['P_Value'].apply(lambda x: -1*(np.log(x)/10))
plot_df = plot_df.set_index('Proteomics')
plot_df#when df of certain cancer doesnt have gene, not added for that cancer since appending(blank in graph, like nan)

In [ ]:
plot_df.to_csv('heat_map_df.csv')

# create csv that has proteins with pos and neg changes in median

In [ ]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < -0.3:
            hasNeg = True
        if item > 0.3:
            hasPos = True
           
    if hasPos & hasNeg:
        return True
    return False


Create a df with only the differences in median columns. 

In [ ]:
sig_df = sig_df.replace(to_replace = r'_proteomics$', value = '', regex = True)


only_med = sig_df.drop(columns= ['Gbm_P_Value','Hnscc_P_Value','Luad_P_Value','Lscc_P_Value',
                     'Brca_P_Value','Ov_P_Value','En_P_Value','Colon_P_Value'])
only_med.head()

Map the pos and neg differences in median with the HasPosNeg function. Slice out genes meeting the criteria from the long df formatted for the HeatMap function.

In [ ]:
only_med["Pos_Neg"] = only_med.apply(HasPosNeg, axis = 1)

pn = only_med.loc[only_med['Pos_Neg'] == True]
pn_genes = list(pn.index) # list of genes that have pos and neg

get = plot_df.index.isin(pn_genes)
plot_df2 = plot_df[get] # Keep genes with pos and neg
plot_df2.Cancer.unique()

In [ ]:
plot_df2.to_csv('pos_neg_df.csv')